# Retail Online Store

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from datetime import datetime

In [89]:
time_format = "%m/%d/%Y %H:%M"

df = pd.read_csv('files/online_retail.csv', parse_dates=['InvoiceDate'], date_format = time_format)

In [90]:
df.head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Cleaning

### Drop Innecesary columns

The dataframe already has an index columns so i'm going to get rid of it

In [91]:
df = df.drop(columns='index')

### Creating features

I'm going to split the ``InvoiceDate`` into 2 features: ``Date`` and ``Hour``

In [92]:
df['Hour'] = df['InvoiceDate'].dt.hour
df['Date'] = df['InvoiceDate'].dt.floor('D')

And re-ordering the dataframe

In [93]:
cols = list(df.columns.values)
cols

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'Hour',
 'Date']

In [94]:
df = df[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'Date', 'Hour', 'UnitPrice', 'CustomerID', 'Country',]]
df.head()

,InvoiceNo,StockCode,Description,Quantity,Date,Hour,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,8,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,8,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,8,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,8,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,8,3.39,17850.0,United Kingdom


In [95]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'Date', 'Hour',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [96]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
Date           datetime64[ns]
Hour                    int32
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [97]:
len(df)

541909

In [98]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
Date                0
Hour                0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

### Filling null data

It won't be usefull to erase 135080 data points from 541909 that it is about 24,92% of the data. Also, the columns that has missing data are from ``CustomerID`` and ``Description``, we can populate the ``Description`` table with the ``StockCode`` after we find the relation between both columns, and ``CustomerID`` can be replaced with some value in order to identify those Invoices.

In [99]:
df['CustomerID'] = df['CustomerID'].fillna(0_000_000_000) # 10 Zeroes

In [100]:
df['Description'] = df['Description'].fillna('NO DESCRIPTION')

In [101]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,Date,Hour,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,8,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,8,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,8,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,8,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,8,3.39,17850.0,United Kingdom


In [102]:
df_retail.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
Date           0
Hour           0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Duplicated Data

In [118]:
duplicated = df[df.duplicated(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID'], keep= False)]
duplicated = duplicated.sort_values(by= ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID'], ascending= [False,False,False,False,False,False])
duplicated

,InvoiceNo,StockCode,Description,Quantity,Date,Hour,UnitPrice,CustomerID,Country
529980,C580764,22667,RECIPE BOX RETROSPOT,-12,2011-12-06,10,2.95,14562.0,United Kingdom
529981,C580764,22667,RECIPE BOX RETROSPOT,-12,2011-12-06,10,2.95,14562.0,United Kingdom
461407,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,2011-11-13,11,0.55,17838.0,United Kingdom
461408,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,2011-11-13,11,0.55,17838.0,United Kingdom
440148,C574510,22360,GLASS JAR ENGLISH CONFECTIONERY,-1,2011-11-04,13,2.95,15110.0,United Kingdom
...,...,...,...,...,...,...,...,...,...
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01,11,2.10,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01,11,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01,11,4.95,17908.0,United Kingdom
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01,11,1.25,17908.0,United Kingdom


There is duplicated data, so i'm going to perform a dropping of those points keeping the first ocurrence

In [120]:
df.drop_duplicates(subset=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID'], keep= 'first',inplace = True)

In [123]:
df['CustomerID'].nunique()
df['StockCode'].nunique()

4070